In [1]:
import cdms2 as cdms
import MV2 as MV
import cdtime,cdutil,genutil
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import string
import glob
import scipy.stats as stats
# Local solution
# If running remotely, uncomment the following code:
# %%bash
# git clone https://github.com/katemarvel/CMIP5_tools
# import CMIP5_tools as cmip5
import sys,os
sys.path.append("/Users/kmarvel/Google Drive/python-utils")

import CMIP5_tools as cmip5
import DA_tools
import Plotting

from eofs.cdms import Eof
from eofs.multivariate.cdms import MultivariateEof
%matplotlib inline


In [2]:
central_plains=cdutil.region.domain(longitude=(-105,-92),latitude=(32,46))
southwest=cdutil.region.domain(longitude=(-125,-105),latitude=(32,41))


In [3]:
#sshfs mout to kdm2144@dester.ldeo.columbia.edu:/home/kdm2144/ 
#datadirec="/Users/kmarvel/Documents/DATA/dester/"
datadirec="/Volumes/RAVEN/CMIP6/cmip6rawyr/"
writedirec="/Users/kmarvel/Documents/DATA/dester/regional_averages/"
fixedvardirec="/Users/kmarvel/Documents/DATA/dester/fixedvar/"

In [4]:
variables=[x.split("/")[-1] for x in glob.glob(datadirec+"*")]


###### LOOP OVER ALL VARIABLES #####
for variable in variables:
    #print(variable)
    cmd = "mkdir "+writedirec+"/central_plains/"+variable
    os.system(cmd)

    cmd = "mkdir "+writedirec+"/southwest/"+variable
    os.system(cmd)
    #modeldirs=glob.glob(datadirec+experiment+"/"+variable+"/*")
    modeldirs=glob.glob(datadirec+variable+"/*")
    ### LOOP OVER ALL MODELS
    for direc in modeldirs:
        model=direc.split("/")[-1]
        #print(model)
        allfiles=glob.glob(direc+"/*"+variable+"*")
        experiments=np.unique([x.split(".")[1] for x in allfiles])
        ###### LOOP OVER ALL EXPERIMENTS #####
        for experiment in experiments:
            #print(experiment)
            cp_writedirec=writedirec+"central_plains/"+variable+"/"+experiment+"/"
            cmd="mkdir "+cp_writedirec
            os.system(cmd)
            sw_writedirec=writedirec+"southwest/"+variable+"/"+experiment+"/"
            cmd="mkdir "+sw_writedirec
            os.system(cmd)
            allfiles_experiment=glob.glob(direc+"/"+variable+"."+experiment+".*")
            rips=np.unique([x.split(".")[-3] for x in allfiles_experiment])

            landthresh=1
            #Get the land fraction
            landfiles=glob.glob(fixedvardirec+"sftlf*"+model+".*")
            

            if len(landfiles)==1:
                fland=cdms.open(landfiles[0])
                landfrac=fland("sftlf")
                fland.close()
            else:
                #print("can't find land fraction file for", model)
                #print(landfiles)
                continue
            #print(direc)



            ###### LOOP OVER ALL RIPS #####
            for rip in rips:
                #print(rip)
                cp_writename=cp_writedirec+variable+"."+experiment+"."+model+"."+rip+".nc"
                if (cp_writename in glob.glob(cp_writedirec+"*")):
                    #print("Already done")
                    continue
                    
                yearcheck=[]
                #ripfiles=sorted(glob.glob(direc+"/*"+rip+"*"))
                ripfiles=np.sort(glob.glob(direc+"/"+variable+"."+experiment+"."+model+"."+rip+"*"))
                print(variable+"."+experiment+"."+model+"."+rip+"*")
                L=len(ripfiles)
                i=0
                ripfile=ripfiles[i]
                frip=cdms.open(ripfile)
                data=frip(variable)
                frip.close()
                if data.shape[1:]!=landfrac.shape:
                    print ("land mask wrong shape for "+variable+"."+experiment+"."+model+"."+rip)
                    continue
                latax=data.getLatitude()
                lonax=data.getLongitude()
                tax=np.arange(12)
                fpath,fexpt,fmodel,frip,fyear,fnc=ripfile.split(".")
                CP=MV.zeros(L*12)
                SW=MV.zeros(L*12)
                #full[12*i:12*(i+1)]=data
                CP[12*i:12*(i+1)]=cdutil.averager(landdata(central_plains),axis='xy')
                SW[12*i:12*(i+1)]=cdutil.averager(landdata(southwest),axis='xy')
                
                yearcheck+=[float(fyear)]
                for i in range(L)[1:]:

                    ripfile=ripfiles[i]
                    
                    f=cdms.open(ripfile)
                    data=f(variable)
                        
                    landdata=MV.masked_where(np.repeat(landfrac.asma()[np.newaxis],12,axis=0)<landthresh,data)
                    f.close()
                    
                    fpath,fexpt,fmodel,frip,fyear,fnc=ripfile.split(".")
                    CP[12*i:12*(i+1)]=cdutil.averager(landdata(central_plains),axis='xy')
                    SW[12*i:12*(i+1)]=cdutil.averager(landdata(southwest),axis='xy')
                    yearcheck+=[float(fyear)]
                tax=cdms.createAxis(np.arange(L*12))
                tax.designateTime()
                tax.units='months since '+str(yearcheck[0])+'-1-1'
                CP.setAxis(0,tax)
                
                SW.setAxis(0,tax)
                CP.id=variable
                SW.id=variable
                cp_writename=cp_writedirec+variable+"."+experiment+"."+model+"."+rip+".nc"
                fcpw=cdms.open(cp_writename,"w")    
                fcpw.write(CP)
                fcpw.close()
                sw_writename=sw_writedirec+variable+"."+experiment+"."+model+"."+rip+".nc"
                fsww=cdms.open(sw_writename,"w")    
                fsww.write(SW)
                fsww.close()  

mrsos.piControl.E3SM-1-0.r1i1p1f1*
land mask wrong shape for mrsos.piControl.E3SM-1-0.r1i1p1f1
mrsos.historical.GFDL-CM4.r1i1p1f1*


/Users/kmarvel/anaconda3/envs/cdat_plus/lib/python3.6/site-packages/numpy/ma/core.py:3169: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  dout = self.data[indx]
/Users/kmarvel/anaconda3/envs/cdat_plus/lib/python3.6/site-packages/numpy/ma/core.py:3201: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  mout = _mask[indx]


mrsos.piControl.GFDL-CM4.r1i1p1f1*
mrsos.ssp245.GFDL-CM4.r1i1p1f1*
mrsos.ssp585.GFDL-CM4.r1i1p1f1*
mrsos.historical.GFDL-ESM4.r1i1p1f1*
mrsos.piControl.GFDL-ESM4.r1i1p1f1*
mrsos.ssp126.GFDL-ESM4.r1i1p1f1*
mrsos.ssp245.GFDL-ESM4.r1i1p1f1*
mrsos.ssp370.GFDL-ESM4.r1i1p1f1*
mrsos.ssp585.GFDL-ESM4.r1i1p1f1*
mrsos.historical.GISS-E2-1-G.r10i1p1f1*
mrsos.historical.GISS-E2-1-G.r1i1p1f1*
mrsos.historical.GISS-E2-1-G.r2i1p1f1*
mrsos.historical.GISS-E2-1-G.r3i1p1f1*
mrsos.historical.GISS-E2-1-G.r4i1p1f1*
mrsos.historical.GISS-E2-1-G.r5i1p1f1*
mrsos.historical.GISS-E2-1-G.r6i1p1f1*
mrsos.historical.GISS-E2-1-G.r7i1p1f1*
mrsos.historical.GISS-E2-1-G.r8i1p1f1*
mrsos.historical.GISS-E2-1-G.r9i1p1f1*
mrsos.piControl.GISS-E2-1-G.r102i1p1f1*
mrsos.piControl.GISS-E2-1-G.r1i1p1f1*
mrsos.historical.GISS-E2-1-H.r10i1p1f1*
mrsos.historical.GISS-E2-1-H.r1i1p1f1*
mrsos.historical.GISS-E2-1-H.r2i1p1f1*
mrsos.historical.GISS-E2-1-H.r3i1p1f1*
mrsos.historical.GISS-E2-1-H.r4i1p1f1*
mrsos.historical.GISS-E2-1-H.r5

pr.ssp370.CanESM5.r2i1p1f1*
pr.ssp370.CanESM5.r2i1p2f1*
pr.ssp370.CanESM5.r3i1p1f1*
pr.ssp370.CanESM5.r3i1p2f1*
pr.ssp370.CanESM5.r4i1p1f1*
pr.ssp370.CanESM5.r4i1p2f1*
pr.ssp370.CanESM5.r5i1p1f1*
pr.ssp370.CanESM5.r5i1p2f1*
pr.ssp370.CanESM5.r6i1p1f1*
pr.ssp370.CanESM5.r6i1p2f1*
pr.ssp370.CanESM5.r7i1p1f1*
pr.ssp370.CanESM5.r7i1p2f1*
pr.ssp370.CanESM5.r8i1p1f1*
pr.ssp370.CanESM5.r8i1p2f1*
pr.ssp370.CanESM5.r9i1p1f1*
pr.ssp370.CanESM5.r9i1p2f1*
pr.ssp585.CanESM5.r10i1p1f1*
pr.ssp585.CanESM5.r10i1p2f1*
pr.ssp585.CanESM5.r1i1p1f1*
pr.ssp585.CanESM5.r1i1p2f1*
pr.ssp585.CanESM5.r2i1p1f1*
pr.ssp585.CanESM5.r2i1p2f1*
pr.ssp585.CanESM5.r3i1p1f1*
pr.ssp585.CanESM5.r3i1p2f1*
pr.ssp585.CanESM5.r4i1p1f1*
pr.ssp585.CanESM5.r4i1p2f1*
pr.ssp585.CanESM5.r5i1p1f1*
pr.ssp585.CanESM5.r5i1p2f1*
pr.ssp585.CanESM5.r6i1p1f1*
pr.ssp585.CanESM5.r6i1p2f1*
pr.ssp585.CanESM5.r7i1p1f1*
pr.ssp585.CanESM5.r7i1p2f1*
pr.ssp585.CanESM5.r8i1p1f1*
pr.ssp585.CanESM5.r8i1p2f1*
pr.ssp585.CanESM5.r9i1p1f1*
pr.ssp585.CanESM5.

pr.ssp126.UKESM1-0-LL.r8i1p1f2*
pr.ssp245.UKESM1-0-LL.r1i1p1f2*
pr.ssp245.UKESM1-0-LL.r2i1p1f2*
pr.ssp245.UKESM1-0-LL.r3i1p1f2*
pr.ssp245.UKESM1-0-LL.r4i1p1f2*
pr.ssp245.UKESM1-0-LL.r8i1p1f2*
pr.ssp370.UKESM1-0-LL.r1i1p1f2*
pr.ssp370.UKESM1-0-LL.r2i1p1f2*
pr.ssp370.UKESM1-0-LL.r3i1p1f2*
pr.ssp370.UKESM1-0-LL.r4i1p1f2*
pr.ssp370.UKESM1-0-LL.r8i1p1f2*
pr.ssp585.UKESM1-0-LL.r1i1p1f2*
pr.ssp585.UKESM1-0-LL.r2i1p1f2*
pr.ssp585.UKESM1-0-LL.r3i1p1f2*
pr.ssp585.UKESM1-0-LL.r4i1p1f2*
pr.ssp585.UKESM1-0-LL.r8i1p1f2*
prsn.historical.BCC-ESM1.r1i1p1f1*
prsn.historical.BCC-ESM1.r2i1p1f1*
prsn.historical.BCC-ESM1.r3i1p1f1*
prsn.historical.CanESM5.r10i1p1f1*
prsn.historical.CanESM5.r10i1p2f1*
prsn.historical.CanESM5.r11i1p1f1*
prsn.historical.CanESM5.r11i1p2f1*
prsn.historical.CanESM5.r12i1p1f1*
prsn.historical.CanESM5.r12i1p2f1*
prsn.historical.CanESM5.r13i1p1f1*
prsn.historical.CanESM5.r13i1p2f1*
prsn.historical.CanESM5.r14i1p1f1*
prsn.historical.CanESM5.r14i1p2f1*
prsn.historical.CanESM5.r15i1p1f1

prsn.historical.MIROC-ES2L.r3i1p1f2*
prsn.ssp245.MIROC-ES2L.r1i1p1f2*
prsn.ssp370.MIROC-ES2L.r1i1p1f2*
prsn.ssp585.MIROC-ES2L.r1i1p1f2*
prsn.historical.MIROC6.r10i1p1f1*
prsn.historical.MIROC6.r1i1p1f1*
prsn.historical.MIROC6.r2i1p1f1*
prsn.historical.MIROC6.r3i1p1f1*
prsn.historical.MIROC6.r4i1p1f1*
prsn.historical.MIROC6.r5i1p1f1*
prsn.historical.MIROC6.r6i1p1f1*
prsn.historical.MIROC6.r7i1p1f1*
prsn.historical.MIROC6.r8i1p1f1*
prsn.historical.MIROC6.r9i1p1f1*
prsn.ssp245.MIROC6.r1i1p1f1*
prsn.ssp245.MIROC6.r2i1p1f1*
prsn.ssp245.MIROC6.r3i1p1f1*
prsn.ssp585.MIROC6.r1i1p1f1*
prsn.ssp585.MIROC6.r2i1p1f1*
prsn.ssp585.MIROC6.r3i1p1f1*
prsn.historical.MRI-ESM2-0.r1i1p1f1*
prsn.historical.MRI-ESM2-0.r2i1p1f1*
prsn.historical.MRI-ESM2-0.r3i1p1f1*
prsn.historical.MRI-ESM2-0.r4i1p1f1*
prsn.historical.MRI-ESM2-0.r5i1p1f1*
prsn.ssp245.MRI-ESM2-0.r1i1p1f1*
prsn.ssp245.MRI-ESM2-0.r2i1p1f1*
prsn.ssp245.MRI-ESM2-0.r3i1p1f1*
prsn.ssp245.MRI-ESM2-0.r4i1p1f1*
prsn.ssp245.MRI-ESM2-0.r5i1p1f1*
prsn.ssp5

tas.ssp585.CNRM-ESM2-1.r4i1p1f2*
tas.ssp585.CNRM-ESM2-1.r5i1p1f2*
tas.historical.GFDL-CM4.r1i1p1f1*
tas.ssp245.GFDL-CM4.r1i1p1f1*
tas.ssp585.GFDL-CM4.r1i1p1f1*
tas.historical.GFDL-ESM4.r1i1p1f1*
tas.ssp126.GFDL-ESM4.r1i1p1f1*
tas.ssp245.GFDL-ESM4.r1i1p1f1*
tas.ssp370.GFDL-ESM4.r1i1p1f1*
tas.ssp585.GFDL-ESM4.r1i1p1f1*
tas.historical.GISS-E2-1-G.r10i1p1f1*
tas.historical.GISS-E2-1-G.r1i1p1f1*
tas.historical.GISS-E2-1-G.r2i1p1f1*
tas.historical.GISS-E2-1-G.r3i1p1f1*
tas.historical.GISS-E2-1-G.r4i1p1f1*
tas.historical.GISS-E2-1-G.r5i1p1f1*
tas.historical.GISS-E2-1-G.r6i1p1f1*
tas.historical.GISS-E2-1-G.r7i1p1f1*
tas.historical.GISS-E2-1-G.r8i1p1f1*
tas.historical.GISS-E2-1-G.r9i1p1f1*
tas.historical.GISS-E2-1-H.r10i1p1f1*
tas.historical.GISS-E2-1-H.r1i1p1f1*
tas.historical.GISS-E2-1-H.r2i1p1f1*
tas.historical.GISS-E2-1-H.r3i1p1f1*
tas.historical.GISS-E2-1-H.r4i1p1f1*
tas.historical.GISS-E2-1-H.r5i1p1f1*
tas.historical.GISS-E2-1-H.r6i1p1f1*
tas.historical.GISS-E2-1-H.r7i1p1f1*
tas.historica